In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import gc

from typing import Dict, Any, Union

from pathlib import Path
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn

import torch.utils.data as D
from torch.utils.data.dataset import Dataset, IterableDataset
from torch.utils.data.dataloader import DataLoader

from transformers import AutoModel, AutoTokenizer
from transformers import PreTrainedModel

from tqdm.notebook import tqdm

from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

### Folders and Dataframes

In [ ]:
DATA_PATH = Path('/kaggle/input/commonlitreadabilityprize/')
assert DATA_PATH.exists()
MODELS_PATH_BART = Path('/kaggle/input/commonlitbartlargelight/best_models')
assert MODELS_PATH_BART.exists()
MODELS_PATH_ROBERTA_LARGE = Path('/kaggle/input/commonlit-roberta-large-light/best_models')
assert MODELS_PATH_ROBERTA_LARGE.exists()
MODELS_PATH_ROBERTA_BALANCED = Path('/kaggle/input/commonlit-roberta-base-light-balanced/best_models')
assert MODELS_PATH_ROBERTA_BALANCED.exists()
MODELS_PATH_ROBERTA_LIGHT_2 = Path('/kaggle/input/commonlit-roberta-large-light-2/best_models')
assert MODELS_PATH_ROBERTA_LIGHT_2.exists()
MODELS_PATH_ROBERTA_BASE_LIGHT = Path('/kaggle/input/commonlit-roberta-base-light/best_models')
assert MODELS_PATH_ROBERTA_BASE_LIGHT.exists()
MODELS_PATH_DEBERTA_LARGE = Path('/kaggle/input/commonlit-deberta-large/best_models')
assert MODELS_PATH_DEBERTA_LARGE.exists()
MODELS_PATH_T5_LARGE = Path('/kaggle/input/commonlit-t5-large/best_models')
assert MODELS_PATH_T5_LARGE.exists()
MODELS_PATH_ELECTRA_LARGE = Path('/kaggle/input/commonlit-electra-large-discriminator/best_models')
assert MODELS_PATH_T5_LARGE.exists()
MODELS_PATH_DEBERTA_UNIFORM = Path('/kaggle/input/commonlit-deberta-large-uniform/best_models')
assert MODELS_PATH_DEBERTA_UNIFORM.exists()
MODELS_PATH_DEBERTA_XLARGE = Path('/kaggle/input/commonlit-deberta-xlarge/best_models')
assert MODELS_PATH_DEBERTA_XLARGE.exists()

In [ ]:
train_df = pd.read_csv(DATA_PATH/'train.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [ ]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

### Configuration

In [ ]:
def assert_path(path):
    assert Path(path).exists(), f"{path} is missing"

In [ ]:
class CONFIG():
    model_name = 'multi-models'
    batch_size = 48
    max_len = 256
    save_dir = f'trained/{model_name}'
    num_workers = 2
    epochs = 20
    pretrained_bart_lm_model = '/kaggle/input/commonlitbartlargelight/lm'
    pretrained_roberta_large_lm_model = '/kaggle/input/commonlit-roberta-large-light/lm'
    pretrained_roberta_base_light_lm_model = '/kaggle/input/commonlit-roberta-base-light/lm'
    pretrained_roberta_light_balanced_lm_model = '/kaggle/input/commonlit-roberta-base-light-balanced/lm'
    pretrained_roberta_light_2_lm_model = '/kaggle/input/commonlit-roberta-large-light-2/lm'
    pretrained_deberta_large_lm_model = '/kaggle/input/commonlit-deberta-large/lm'
    pretrained_t5_large_lm_model = '/kaggle/input/commonlit-t5-large/lm'
    pretrained_electra_large_lm_model = str(MODELS_PATH_ELECTRA_LARGE.parent/'lm')
    pretrained_deberta_uniform_model = str(MODELS_PATH_DEBERTA_UNIFORM.parent/'lm')
    pretrained_deberta_xlarge_model = str(MODELS_PATH_DEBERTA_XLARGE.parent/'lm')
    assert_path(pretrained_bart_lm_model)
    assert_path(pretrained_roberta_large_lm_model)
    assert_path(pretrained_roberta_base_light_lm_model)
    assert_path(pretrained_roberta_light_balanced_lm_model)
    assert_path(pretrained_roberta_light_2_lm_model)
    assert_path(pretrained_deberta_large_lm_model)
    assert_path(pretrained_t5_large_lm_model)
    assert_path(pretrained_electra_large_lm_model)
    assert_path(pretrained_deberta_uniform_model)
    assert_path(pretrained_deberta_xlarge_model)
    n_folds = 5
    model_offset = 0
    model_limit = 5
    svm_c = 10
    svm_kernels = ['rbf']
    
cfg = CONFIG()

### Read Existing Models

In [ ]:
models_map = {
    'deberta_xlarge' : {'model': 'CommonLitDebertaXLargeModel', 'tokenizer_path': MODELS_PATH_DEBERTA_XLARGE, 'weight': 0.15},
    'deberta_uniform' : {'model': 'CommonLitLightModel', 'tokenizer_path': MODELS_PATH_DEBERTA_UNIFORM, 'weight': 0.16},
    'electra_large' : {'model': 'CommonLitElectraModel', 'tokenizer_path': MODELS_PATH_ELECTRA_LARGE, 'weight': 0.13},
#     't5_large' : {'model': 'CommonLitT5Model', 'tokenizer_path': MODELS_PATH_T5_LARGE, 'weight': 0.11},
    'roberta_deberta_large': {'model': 'CommonLitLightModel', 'tokenizer_path': MODELS_PATH_DEBERTA_LARGE, 'weight': 0.18},
    'roberta_base_light' : {'model': 'CommonLitLightModel', 'tokenizer_path': MODELS_PATH_ROBERTA_BASE_LIGHT, 'weight': 0.11},
    'roberta_light_balanced' : {'model': 'CommonLitLightModel', 'tokenizer_path': MODELS_PATH_ROBERTA_BALANCED, 'weight': 0.11},
#     'bart' : {'model': 'CommonLitBartModel', 'tokenizer_path': MODELS_PATH_BART, 'weight': 0.11},
#     'roberta_light_2': {'model': 'CommonLitLightModel', 'tokenizer_path': MODELS_PATH_ROBERTA_LIGHT_2, 'weight': 0.11},
    'roberta_large' : {'model': 'CommonLitRobertaLargeModel', 'tokenizer_path': MODELS_PATH_ROBERTA_LARGE, 'weight': 0.16}
}   

In [ ]:
cfg.model_avg_weights = [i[1]['weight'] for i in models_map.items()]
cfg.model_avg_weights

In [ ]:
np.abs(np.array(cfg.model_avg_weights).sum() - 1.0)

In [ ]:
model_items = models_map.items()

if not (np.abs(np.array(cfg.model_avg_weights).sum() - 1.0) < 1e-8):
    default_weight = 1 / len(model_items)
    for k, v in model_items:
        v['weight'] = default_weight
        
cfg.model_avg_weights = [i[1]['weight'] for i in models_map.items()]
cfg.model_avg_weights

In [ ]:
assert len(models_map) == len(cfg.model_avg_weights), f"Length of models map ({len(models_map)}) is not the same as averaging weights ({len(cfg.model_avg_weights)})"
assert np.abs(np.array(cfg.model_avg_weights).sum() - 1.0) < 1e-8, f'{np.sum(cfg.model_avg_weights)}'

In [ ]:
!ls {MODELS_PATH_BART}

In [ ]:
!ls {MODELS_PATH_ROBERTA_LARGE}

In [ ]:
!ls {MODELS_PATH_ROBERTA_BALANCED}

In [ ]:
!ls {MODELS_PATH_DEBERTA_LARGE}

In [ ]:
!ls {MODELS_PATH_ROBERTA_LIGHT_2}

In [ ]:
!ls {MODELS_PATH_T5_LARGE}

In [ ]:
def get_cls_light_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            _, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [ ]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [ ]:
class AttentionLargeHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [ ]:
from transformers import AutoModelForSequenceClassification

class CommonLitElectraModel(nn.Module):
    def __init__(self, config_path=cfg.pretrained_electra_large_lm_model):
        super(CommonLitElectraModel, self).__init__()
        config = AutoConfig.from_pretrained(config_path)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(config_path, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
from transformers import T5EncoderModel

class CommonLitT5Model(nn.Module):
    def __init__(self):
        super(CommonLitT5Model, self).__init__()
        config = AutoConfig.from_pretrained(cfg.pretrained_t5_large_lm_model)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = T5EncoderModel.from_pretrained(cfg.pretrained_t5_large_lm_model, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
from transformers import PreTrainedModel, AutoConfig

class CommonLitBartModel(nn.Module):
    def __init__(self):
        super(CommonLitBartModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.pretrained_bart_lm_model)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_bart_lm_model, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
class AttentionSimpleHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        
        attention_weights = torch.softmax(score, dim=1)
        context_vector = attention_weights * features
        context_vector = torch.sum(context_vector, dim=1)
        return context_vector

In [ ]:
def get_embeddings(dl, transformer_model):
    cls_embeddings = []
    hidden_size = 1024 if str(type(inference_model)).find('Bart') > -1 else 768
    attention_head = AttentionSimpleHead(hidden_size, hidden_size, 1).cuda()
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda(), output_hidden_states=True)
            last_hidden_state = output['last_hidden_state']
            hidden_state_name = 'decoder_hidden_states' if 'decoder_hidden_states' in output else 'hidden_states'
            previous_hidden_state = output[hidden_state_name][-2]
            cls_state = last_hidden_state[:,0,:].detach().cpu().numpy()
            cls_state_2 = attention_head(last_hidden_state).detach().cpu().numpy()
            cls_state_3 = previous_hidden_state[:,0,:].detach().cpu().numpy()
            concatenated = np.average([cls_state, cls_state_2, cls_state_3], axis=0)
            cls_embeddings.extend(concatenated)
    return np.array(cls_embeddings)

In [ ]:
class CommonLitRobertaLargeModel(nn.Module):
    def __init__(self):
        super(CommonLitRobertaLargeModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.pretrained_roberta_large_lm_model)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_roberta_large_lm_model, config=config)
        self.attention = AttentionLargeHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
class CommonLitRobertaModel(PreTrainedModel):
    def __init__(self):
        super(PreTrainedModel, self).__init__()
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_roberta_lm_model)
        self.drop = nn.Dropout(0.5)
        self.config = AutoConfig.from_pretrained(cfg.pretrained_roberta_lm_model)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)
        self.out = nn.Linear(self.config.hidden_size, 1)
        self.transformer_model.get_cls_embeddings = self.get_cls_embeddings
#         self._init_weights(self.layer_norm)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        if isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer_model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), output_hidden_states=False)
        x = transformer_out.pooler_output
#         x = transformer_out.last_hidden_state[:, 0, :] # N, C, X
        x = self.layer_norm(x)
        x = self.drop(x)
        x = self.out(x)
        return x
    
    def floating_point_ops(self, inputs: Dict[str, Union[torch.Tensor, Any]]):
        """
        For models that inherit from :class:`~transformers.PreTrainedModel`, uses that method to compute the number of
        floating point operations for every backward + forward pass. If using another model, either implement such a
        method in the model or subclass and override this method.
        Args:
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
        Returns:
            :obj:`int`: The number of floating-point operations.
        """
        return 0
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_embeddings(dl, transformer_model)

In [ ]:
class CommonLitRobertaStandardizedModel(PreTrainedModel):
    def __init__(self):
        super(PreTrainedModel, self).__init__()
        self.transformer_model = AutoModel.from_pretrained(cfg.pretrained_roberta_standardized_lm_model)
        self.drop = nn.Dropout(0.5)
        self.config = AutoConfig.from_pretrained(cfg.pretrained_roberta_standardized_lm_model)
        self.layer_norm = nn.LayerNorm(self.config.hidden_size)
        self.out = nn.Linear(self.config.hidden_size, 1)
        self._init_weights(self.layer_norm)
        self.transformer_model.get_cls_embeddings = self.get_cls_embeddings
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        if isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def forward(self, input_ids, attention_mask):
        transformer_out = self.transformer_model(input_ids=input_ids.squeeze(), attention_mask=attention_mask.squeeze(), 
                                                 output_hidden_states=True)
#         x = transformer_out.pooler_output
#         x = transformer_out.last_hidden_state[:, 0, :] # N, C, X
        x = torch.mean(transformer_out.hidden_states[-2], axis=1)
#         x2 = torch.mean(transformer_out.encoder_hidden_states[-2], axis=1)
#         x = torch.cat([x, x2], axis=1)
        x = self.layer_norm(x)
        x = self.drop(x)
        x = self.out(x)
        return x
    
    def floating_point_ops(self, inputs: Dict[str, Union[torch.Tensor, Any]]):
        """
        For models that inherit from :class:`~transformers.PreTrainedModel`, uses that method to compute the number of
        floating point operations for every backward + forward pass. If using another model, either implement such a
        method in the model or subclass and override this method.
        Args:
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.
        Returns:
            :obj:`int`: The number of floating-point operations.
        """
        return 0
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_embeddings(dl, transformer_model)

In [ ]:
class CommonLitLightModel(nn.Module):
    def __init__(self, lm_model=cfg.pretrained_roberta_large_lm_model):
        super(CommonLitLightModel, self).__init__()
        config = AutoConfig.from_pretrained(lm_model)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(lm_model, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
from transformers import AutoModelForSequenceClassification

class CommonLitDebertaXLargeModel(nn.Module):
    def __init__(self):
        super(CommonLitDebertaXLargeModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.pretrained_deberta_xlarge_model)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModelForSequenceClassification.from_pretrained(cfg.pretrained_deberta_xlarge_model, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['hidden_states']
        last_layer_hidden_states = hidden_states[-1]
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector
    
    def get_cls_embeddings(self, dl, transformer_model):
        return get_cls_light_embeddings(dl, transformer_model)

In [ ]:
def load_model(model_name='CommonLitBartModel', model_path=MODELS_PATH_BART, i=0):
    if model_path in [MODELS_PATH_ROBERTA_LARGE, MODELS_PATH_ROBERTA_LIGHT_2, MODELS_PATH_BART, 
                      MODELS_PATH_DEBERTA_LARGE, MODELS_PATH_T5_LARGE, MODELS_PATH_ELECTRA_LARGE,
                     MODELS_PATH_DEBERTA_UNIFORM, MODELS_PATH_DEBERTA_XLARGE]:
        i = i + 1
    print(f'Model {model_name} {i}')
    if model_path == MODELS_PATH_ROBERTA_LIGHT_2:
        inference_model = CommonLitLightModel(cfg.pretrained_roberta_light_2_lm_model)
    elif model_path == MODELS_PATH_ROBERTA_BALANCED:
        inference_model = CommonLitLightModel(cfg.pretrained_roberta_light_balanced_lm_model)
    elif model_path == MODELS_PATH_ROBERTA_BASE_LIGHT:
        inference_model = CommonLitLightModel(cfg.pretrained_roberta_base_light_lm_model)
    elif model_path == MODELS_PATH_DEBERTA_LARGE:
        inference_model = CommonLitLightModel(cfg.pretrained_deberta_large_lm_model)
    elif model_path == MODELS_PATH_T5_LARGE:
        inference_model = CommonLitT5Model()
    elif model_path == MODELS_PATH_ELECTRA_LARGE:
        inference_model = CommonLitElectraModel(config_path=cfg.pretrained_electra_large_lm_model)
    elif model_path == MODELS_PATH_DEBERTA_UNIFORM:
        inference_model = CommonLitLightModel(cfg.pretrained_deberta_uniform_model)
    elif model_path == MODELS_PATH_DEBERTA_XLARGE:
        inference_model = CommonLitDebertaXLargeModel()
    else:
        inference_model = eval(f'{model_name}()')
    inference_model = inference_model.cuda()
    bin_file = f'{i}_pytorch_model.bin'
    print(f'Loading model from {model_path/bin_file}')
    inference_model.load_state_dict(torch.load(str(model_path/bin_file)))
    inference_model.eval();
    return inference_model

### DataSet and Tokenizers

In [ ]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [ ]:
from transformers import RobertaTokenizer, BartTokenizer, DebertaTokenizer, T5Tokenizer, ElectraTokenizer

def load_tokenizer(model_path=MODELS_PATH_BART, i=0):
    if model_path in [MODELS_PATH_ROBERTA_LARGE, MODELS_PATH_ROBERTA_LIGHT_2, MODELS_PATH_DEBERTA_LARGE, 
                      MODELS_PATH_BART, MODELS_PATH_T5_LARGE, MODELS_PATH_ELECTRA_LARGE, 
                      MODELS_PATH_DEBERTA_UNIFORM, MODELS_PATH_DEBERTA_XLARGE]:
        i = i + 1
    tokenizer_path =  model_path/f'tokenizer-{i}'
    if model_path in [MODELS_PATH_ROBERTA_LARGE, MODELS_PATH_ROBERTA_LIGHT_2]:
        print('Tokenizer', tokenizer_path)
        tokenizer = RobertaTokenizer.from_pretrained(tokenizer_path)
    elif model_path == MODELS_PATH_BART:
        tokenizer = BartTokenizer.from_pretrained(tokenizer_path)
    elif model_path == MODELS_PATH_DEBERTA_LARGE:
        tokenizer = DebertaTokenizer.from_pretrained(tokenizer_path)
    elif model_path == MODELS_PATH_T5_LARGE:
        tokenizer = T5Tokenizer.from_pretrained(str(tokenizer_path))
    elif model_path == MODELS_PATH_ELECTRA_LARGE:
        tokenizer = ElectraTokenizer.from_pretrained(str(tokenizer_path))
    elif model_path == MODELS_PATH_DEBERTA_UNIFORM:
        tokenizer = DebertaTokenizer.from_pretrained(str(tokenizer_path))
    elif model_path == MODELS_PATH_DEBERTA_XLARGE:
        tokenizer = DebertaTokenizer.from_pretrained(str(tokenizer_path))
    else:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return tokenizer

In [ ]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.max_len)
    return DataLoader(ds, 
                      batch_size = cfg.batch_size,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

#### Extract Number of Bins

In [ ]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

#### Training

In [ ]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [ ]:
def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

def calc_avg(scores, weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [ ]:
def train_infer(inference_model, tokenizer):
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    if isinstance(inference_model, (CommonLitLightModel, CommonLitRobertaLargeModel, 
                                    CommonLitBartModel, CommonLitT5Model, CommonLitElectraModel, CommonLitDebertaXLargeModel)):
        transformer_model = inference_model
    elif hasattr(inference_model, 'transformer_model'):
        transformer_model = inference_model.transformer_model
    else:
        transformer_model = inference_model
    transformer_model.cuda()
    X = transformer_model.get_cls_embeddings(train_dl, transformer_model)
    y = train_df['target'].values
    X_test = transformer_model.get_cls_embeddings(test_dl, transformer_model)
    kfold = StratifiedKFold(n_splits=cfg.n_folds)
    scores = []
    rmse_scores = []
    for kernel in cfg.svm_kernels:
        print('* Kernel', kernel, ' *')
        kernel_scores = []
        kernel_rmse_scores = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):
            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        scores.append(calc_mean(kernel_scores))
        rmse_scores.append(calc_mean(kernel_rmse_scores))
    
    del X_test
    del X
    del y
    del kfold
    del test_dl
    del train_dl
    
    return scores, rmse_scores

In [ ]:
%%time

final_scores = []
final_rmse_scores = []
all_preds = []
for i in range(cfg.model_offset, cfg.model_limit):
    print(f'\n#### Round {i} ####\n')
    model_scores = []
    model_rmse_scores = []
    for (_, model_data) in models_map.items():
        print(f"#### {i} - {model_data['model']} ####")
        path = model_data['tokenizer_path']
        inference_model = load_model(model_name=model_data['model'], model_path=path, i=i)
        tokenizer = load_tokenizer(model_path=path, i=i)
        scores, rmse_scores = train_infer(inference_model, tokenizer)
        scores = calc_mean(scores)
        all_preds.append(scores)
        rmse_scores = calc_mean(rmse_scores)
        model_scores.append(scores)
        model_rmse_scores.append(rmse_scores)
        print('scores shape', np.array(scores).shape)
        print('rmse_scores shape', np.array(rmse_scores).shape)
        del inference_model
        torch.cuda.empty_cache()
        del tokenizer
        gc.collect()
    final_scores.append(np.average(model_scores, weights=cfg.model_avg_weights, axis=0))
    final_rmse_scores.append(calc_mean(model_rmse_scores))
print('FINAL RMSE score', calc_mean(final_rmse_scores))

#### Ensure the mean of the prediction equals the mean of the training data

Calculate the weights based on the rmse score

In [ ]:
final_rmse_scores, final_scores

In [ ]:
kernel_rmse_score_mean_array = np.array(final_rmse_scores)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

In [ ]:
weights.sum()

In [ ]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_avg(final_scores, weights).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(all_preds).flatten().mean(), np.array(final_scores).mean()

In [ ]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

In [ ]:
sample_df['target'] = final_scores_flat + mean_diff / len(final_scores)
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

In [ ]:
sample_df['target'].mean() # Should be very close to the train mean

In [ ]:
pd.DataFrame(sample_df).to_csv('submission.csv', index=False)

In [ ]:
!cat submission.csv